In [4]:
# modules for webscraping (and cleaning)
import pandas as pd
import requests
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# load api_key
with open('gov_apikey.txt', 'r') as file:
    apikey = file.read()


In [ ]:
# base url
url = "http://api.data.gov/ed/collegescorecard/v1/schools?" # goverment data on colleges

In [ ]:
# Dictionary of all desired fields
year = "latest" # get latest data (if want data from a specific year, and can specify that here)
# first pull: Fall 2022
fields = {
      # School Category
      "School Name": "school.name",
      "School ID": "id",
      "State": "school.state",
      "School Ownership": "school.ownership",
      "Full-time Faculty Rate (%)": "school.ft_faculty_rate",
      "Faculty Average Monthly Salary": "school.faculty_salary",
      # Student Category
      "Student Enrollment Size": year + ".student.size",
      # "Student Enrollment All": year + ".student.enrollment.all", # this field has been discontinued
      "Male Students (%)": year + ".student.demographics.men",
      "Female Students (%)": year + ".student.demographics.women",
      "Retention Rate 4 Yr (%)": year + ".student.retention_rate.four_year.full_time", # this should be labeled as "first:second year retention rate"
      # definition: "the share of full-time and part-time students in the prior year ... who return to the institution after the first year" - Technical Documentation
      # Cost Category
      "Attendance Cost (Academic Year)": year + ".cost.attendance.academic_year",
      # ADDITION: net cost (split up into public and private universities)
      "NetCost_public": year + "avg_net_price.public",
      "NetCost_private": year + "avg_net_price.private",
      # Completion Category
      "150% Completion Rate at 4 Yr (%)": year + ".completion.completion_rate_4yr_150nt",
      # Admissions Category 
      "Admission Rate (%)": year + ".admissions.admission_rate.overall",
      "SAT Average Overall": year + ".admissions.sat_scores.average.overall", # NOTE: this is SAT score equivalent, so should encompass ACT scores as well
      "SAT 75th Percentile Math": year + ".admissions.sat_scores.75th_percentile.math",
      "SAT 75th Percentile Reading": year + ".admissions.sat_scores.75th_percentile.critical_reading",
      "SAT 75th Percentile Writing": year + ".admissions.sat_scores.75th_percentile.writing",
      ## Earnings Category
      # could also add number of students not working and not enrolled after 6 and/or 10 years
      # 6 Years after Enrollment:
      "Mean Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.mean_earnings",
      # add median earnings
      "Median Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.median",
      "Mean Male Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.male_students",
      "Mean Female Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.female_students",
      "Std. Deviation Earning (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.std_dev",
      "Percent of Students Earning >$25K (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.percent_greater_than_25000",
      "Low Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.lowest_tercile",
      "Medium Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.middle_tercile",
      "High Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.highest_tercile",
      # these are mean earnings of students based on their background financial status (low, med, or high income situation/status)
      "Mean Earnings Low (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.lowest_tercile",
      "Mean Earnings Medium (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.middle_tercile",
      "Mean Earnings High (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.highest_tercile",
      # 10 Years after Enrollment:
      "Mean Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.mean_earnings",
      # add median earnings
      "Median Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.median",
      "Mean Male Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.male_students",
      "Mean Female Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.female_students",
      "Std. Deviation Earning (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.std_dev",
      "Percent of Students Earning >$25K (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.percent_greater_than_25000",
      "Low Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.lowest_tercile",
      "Medium Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.middle_tercile",
      "High Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.highest_tercile",
      "Mean Earnings Low (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.lowest_tercile",
      "Mean Earnings Medium (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.middle_tercile",
      "Mean Earnings High (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.highest_tercile"
     }

In [ ]:
# Appending all the fields values together into one "fields_url"
fields_url = ""
for key, val in fields.items():
    fields_url = fields_url + val + ","

# To remove the extra "," at the end of fields_url
fields_url = fields_url[:-1]
fields_url

'school.name,id,school.state,school.ownership,school.ft_faculty_rate,school.faculty_salary,latest.student.size,latest.student.enrollment.all,latest.student.demographics.men,latest.student.demographics.women,latest.student.retention_rate.four_year.full_time,latest.cost.attendance.academic_year,latest.completion.completion_rate_4yr_150nt,latest.admissions.admission_rate.overall,latest.admissions.sat_scores.average.overall,latest.admissions.sat_scores.75th_percentile.math,latest.admissions.sat_scores.75th_percentile.critical_reading,latest.admissions.sat_scores.75th_percentile.writing,latest.earnings.6_yrs_after_entry.working_not_enrolled.mean_earnings,latest.earnings.6_yrs_after_entry.mean_earnings.male_students,latest.earnings.6_yrs_after_entry.mean_earnings.female_students,latest.earnings.6_yrs_after_entry.working_not_enrolled.std_dev,latest.earnings.6_yrs_after_entry.percent_greater_than_25000,latest.earnings.6_yrs_after_entry.working_not_enrolled.income.lowest_tercile,latest.earnings

In [ ]:
# conditional parameters
params = {
    "api_key":apikey,
    "school.degrees_awarded.predominant":"3", # predominantly bachelors degree awarding
    "school.operating":"1" # school operating = True
    
}

In [ ]:
# pull from api using requests and conditional parameters (including api key)
r = requests.get(url, params=params)
print(r.status_code)
# print(r.url)
r.ok

200


True

In [ ]:
# view keys in json file
r.json().keys()

dict_keys(['metadata', 'results'])

In [ ]:
# get and save max page number
max_page_num = r.json()['metadata']['total']//100 + 1

In [ ]:
# test query format for all desired fields
query_url = f'{r.url}&fields={fields_url}&page=0'
print(requests.get(query_url).ok)
# response = requests.get(query_url).json()["results"]

True


In [ ]:
# Construct df via looping through all pages
college_df = []
per_page = 100

for page_num in range(0,max_page_num):
    query_url = f'{r.url}&fields={fields_url}&page={page_num}&_per_page={per_page}'
    response = requests.get(query_url).json()
    
    for i in range(len(response["results"])):
        result_row = {}
        
        for key, val in fields.items(): # set key as var name and value as value
            try:
                result_row[key] = response["results"][i][val]
            except KeyError:
                print(f"{key} key not found")
                
        college_df.append(result_row)

        
college_df = pd.DataFrame(college_df)
# college_df

In [ ]:
# CLEANING
# Update School Ownership to Named Meaning:
# (1: "Public", 2: "Private NonProfit", 3: "Private ForProfit")
college_df.loc[college_df["School Ownership"] == 1, "School Ownership"] = "Public"
college_df.loc[college_df["School Ownership"] == 2, "School Ownership"] = "Private NonProfit"
college_df.loc[college_df["School Ownership"] == 3, "School Ownership"] = "Private ForProfit"

In [ ]:
# Change columns with Percent to Percent Form (*100)
print(college_df.columns)
# 4,8,9,10,12,13,21 are percents

# change these columns to percents
college_df.iloc[:,[4,8,9,10,12,13,22]] = college_df.iloc[:,[4,8,9,10,12,13,22]]*100
college_df.iloc[:,[4,8,9,10,12,13,22]]

Index(['School Name', 'School ID', 'State', 'School Ownership',
       'Full-time Faculty Rate (%)', 'Faculty Average Monthly Salary',
       'Student Enrollment Size', 'Student Enrollment All',
       'Male Students (%)', 'Female Students (%)', 'Retention Rate 4 Yr (%)',
       'Attendance Cost (Academic Year)', '150% Completion Rate at 4 Yr (%)',
       'Admission Rate (%)', 'SAT Average Overall', 'SAT 75th Percentile Math',
       'SAT 75th Percentile Reading', 'SAT 75th Percentile Writing',
       'Mean Earnings (6 Yrs after Entry)',
       'Mean Male Earnings (6 Yrs after Entry)',
       'Mean Female Earnings (6 Yrs after Entry)',
       'Std. Deviation Earning (6 Yrs after Entry)',
       'Percent of Students Earning >$25K (6 Yrs after Entry)',
       'Low Income Students (6 Yrs after Entry)',
       'Medium Income Students (6 Yrs after Entry)',
       'High Income Students (6 Yrs after Entry)',
       'Mean Earnings Low (6 Yrs after Entry)',
       'Mean Earnings Medium (6 Yrs a

,Full-time Faculty Rate (%),Male Students (%),Female Students (%),Retention Rate 4 Yr (%),150% Completion Rate at 4 Yr (%),Admission Rate (%),Percent of Students Earning >$25K (6 Yrs after Entry)
0,99.60,39.78,60.22,54.03,28.66,89.65,45.3
1,76.19,38.16,61.84,86.40,61.17,80.60,66.9
2,67.02,58.91,41.09,81.80,57.14,77.11,68.5
3,67.97,36.05,63.95,62.02,31.77,98.88,39.3
4,77.07,44.17,55.83,87.23,72.14,80.39,69.5
...,...,...,...,...,...,...,...
1984,NaN,10.59,89.41,NaN,NaN,90.91,NaN
1985,82.80,53.67,46.33,86.91,72.81,78.27,NaN
1986,NaN,46.15,53.85,100.00,NaN,100.00,NaN
1987,NaN,0.00,100.00,NaN,NaN,NaN,NaN


In [ ]:
# my dataset!
college_df

,School Name,School ID,State,School Ownership,Full-time Faculty Rate (%),Faculty Average Monthly Salary,Student Enrollment Size,Student Enrollment All,Male Students (%),Female Students (%),...,Mean Male Earnings (10 Yrs after Entry),Mean Female Earnings (10 Yrs after Entry),Std. Deviation Earning (10 Yrs after Entry),Percent of Students Earning >$25K (10 Yrs after Entry),Low Income Students (10 Yrs after Entry),Medium Income Students (10 Yrs after Entry),High Income Students (10 Yrs after Entry),Mean Earnings Low (10 Yrs after Entry),Mean Earnings Medium (10 Yrs after Entry),Mean Earnings High (10 Yrs after Entry)
0,Alabama A & M University,100654,AL,Public,99.60,7599.0,5090.0,None,39.78,60.22,...,38500.0,32600.0,25400.0,0.599,459.0,337.0,166.0,33000.0,37300.0,39500.0
1,University of Alabama at Birmingham,100663,AL,Public,76.19,11380.0,13549.0,None,38.16,61.84,...,57400.0,43200.0,39200.0,0.747,1304.0,911.0,649.0,47000.0,49500.0,49300.0
2,University of Alabama in Huntsville,100706,AL,Public,67.02,9697.0,7825.0,None,58.91,41.09,...,58700.0,46000.0,33600.0,0.779,682.0,454.0,378.0,47000.0,55500.0,55100.0
3,Alabama State University,100724,AL,Public,67.97,7194.0,3603.0,None,36.05,63.95,...,33000.0,28300.0,21400.0,0.528,1519.0,548.0,151.0,29000.0,32500.0,34300.0
4,The University of Alabama,100751,AL,Public,77.07,10349.0,30610.0,None,44.17,55.83,...,59100.0,45100.0,42500.0,0.786,1424.0,1545.0,2024.0,45500.0,51600.0,55800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,Arizona College of Nursing-Phoenix,495457,AZ,Private ForProfit,NaN,NaN,85.0,None,10.59,89.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985,The Pennsylvania State University,495767,PA,Public,82.80,10822.0,73189.0,None,53.67,46.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986,Pathways College,495916,CA,Private NonProfit,NaN,NaN,13.0,None,46.15,53.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987,Provo College-Idaho Falls Campus,496283,ID,Private ForProfit,NaN,4583.0,16.0,None,0.00,100.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# write to csv
college_df.to_csv("./college_df.csv",index=False)

After original cleaning above, further cleaning and preprocessing was found necessary for EDA plots and model training. This cleaning is shown below.

In [5]:
# df = college_df # uncomment this line when ready for new data pull
df = pd.read_csv("./college_df.csv")

In [6]:
# clean up missing data column
print("df shape before:", df.shape)
df = df.dropna(how = 'all',axis = 1) # drop columns with all nas
print("df shape after removing columns of all missing values:", df.shape)

df shape before: (1989, 40)
df shape after removing columns of all missing values: (1989, 39)


In [7]:
# new columns: (not currently used in modeling, so could remove this cell if needed. Will keep for now)
df = df.assign(Female_Majority = df["Female Students (%)"] > 50.00, #Female_Majority female students > 50%
               # Gender Diff for mean earnings
               MeanEarningsGenderDiff10 = df["Mean Male Earnings (10 Yrs after Entry)"]-df["Mean Female Earnings (10 Yrs after Entry)"],
               LowIncRatio_10yr = df['Low Income Students (10 Yrs after Entry)']/df["Student Enrollment Size"],
               HighIncRatio_10yr = df['High Income Students (10 Yrs after Entry)']/df["Student Enrollment Size"],
               LowIncRatio_6yr = df['Low Income Students (6 Yrs after Entry)']/df["Student Enrollment Size"],
               HighIncRatio_6yr = df['High Income Students (6 Yrs after Entry)']/df["Student Enrollment Size"],
               ) 

In [8]:
# change Faculty average salary to a yearly measure
df['Faculty Average Salary'] = df['Faculty Average Monthly Salary']*12
# Fix Percent > 25k for 10 year
df['Percent of Students Earning >$25K (10 Yrs after Entry)'] = df['Percent of Students Earning >$25K (10 Yrs after Entry)']*100
df = df.rename(columns={'Retention Rate 4 Yr (%)': 'RetentionRate_4yr',
                        'Attendance Cost (Academic Year)': 'Attendance Cost',
                       'SAT Average Overall': 'SAT Average (Overall)'})

In [9]:
# get mean earnings vars
# could be useful later ... not currently used
mean_earn_vars = [i for i in list(df.columns) if str(i).startswith("Mean Earnings")]
print(mean_earn_vars)

['Mean Earnings (6 Yrs after Entry)', 'Mean Earnings Low (6 Yrs after Entry)', 'Mean Earnings Medium (6 Yrs after Entry)', 'Mean Earnings High (6 Yrs after Entry)', 'Mean Earnings (10 Yrs after Entry)', 'Mean Earnings Low (10 Yrs after Entry)', 'Mean Earnings Medium (10 Yrs after Entry)', 'Mean Earnings High (10 Yrs after Entry)']


In [10]:
print(list(df.columns))
vars_of_interest = ["School Name", "State", "School Ownership", 'Full-time Faculty Rate (%)', 'Faculty Average Salary', 'Student Enrollment Size',
                    'Attendance Cost', '150% Completion Rate at 4 Yr (%)', 'Admission Rate (%)', 'RetentionRate_4yr', 'Female_Majority', 'SAT Average (Overall)',
                    # 'Percent of Students Earning >$25K (6 Yrs after Entry)', 'Percent of Students Earning >$25K (10 Yrs after Entry)',
                    # 'LowIncRatio_10yr', 'HighIncRatio_10yr', 'LowIncRatio_6yr', 'HighIncRatio_6yr',
                    'Mean Earnings (6 Yrs after Entry)', 'Mean Earnings (10 Yrs after Entry)']
                    # later could also add Median Earnings
                    # could also add mean earnings for low, med, high income students

df_earn_new = df.loc[:,vars_of_interest]
# df_earn_new.to_csv("./college_earnings_for_modeling.csv",index = False)
print(df_earn_new.shape)
df_earn_new.head()

['School Name', 'School ID', 'State', 'School Ownership', 'Full-time Faculty Rate (%)', 'Faculty Average Monthly Salary', 'Student Enrollment Size', 'Male Students (%)', 'Female Students (%)', 'RetentionRate_4yr', 'Attendance Cost', '150% Completion Rate at 4 Yr (%)', 'Admission Rate (%)', 'SAT Average (Overall)', 'SAT 75th Percentile Math', 'SAT 75th Percentile Reading', 'SAT 75th Percentile Writing', 'Mean Earnings (6 Yrs after Entry)', 'Mean Male Earnings (6 Yrs after Entry)', 'Mean Female Earnings (6 Yrs after Entry)', 'Std. Deviation Earning (6 Yrs after Entry)', 'Percent of Students Earning >$25K (6 Yrs after Entry)', 'Low Income Students (6 Yrs after Entry)', 'Medium Income Students (6 Yrs after Entry)', 'High Income Students (6 Yrs after Entry)', 'Mean Earnings Low (6 Yrs after Entry)', 'Mean Earnings Medium (6 Yrs after Entry)', 'Mean Earnings High (6 Yrs after Entry)', 'Mean Earnings (10 Yrs after Entry)', 'Mean Male Earnings (10 Yrs after Entry)', 'Mean Female Earnings (10 Y

,School Name,State,School Ownership,Full-time Faculty Rate (%),Faculty Average Salary,Student Enrollment Size,Attendance Cost,150% Completion Rate at 4 Yr (%),Admission Rate (%),RetentionRate_4yr,Female_Majority,SAT Average (Overall),Mean Earnings (6 Yrs after Entry),Mean Earnings (10 Yrs after Entry)
0,Alabama A & M University,AL,Public,99.60,91188.0,5090.0,23445.0,28.66,89.65,54.03,True,959.0,28400.0,35500.0
1,University of Alabama at Birmingham,AL,Public,76.19,136560.0,13549.0,25542.0,61.17,80.60,86.40,True,1245.0,39400.0,48400.0
2,University of Alabama in Huntsville,AL,Public,67.02,116364.0,7825.0,24861.0,57.14,77.11,81.80,False,1300.0,40300.0,52000.0
3,Alabama State University,AL,Public,67.97,86328.0,3603.0,21892.0,31.77,98.88,62.02,True,938.0,24400.0,30600.0
4,The University of Alabama,AL,Public,77.07,124188.0,30610.0,30016.0,72.14,80.39,87.23,True,1262.0,42400.0,51600.0


In [11]:
# *NOTE*: could do cluster analysis and try to form 3-5 distinct clusters of universities, and then use that university cluster as a feature!
# (could do so quite easily using K means, and clustering based on numeric non-response variables (columns 4-11 in df_earn_new))

In [12]:
df_earn_new.describe()

,Full-time Faculty Rate (%),Faculty Average Salary,Student Enrollment Size,Attendance Cost,150% Completion Rate at 4 Yr (%),Admission Rate (%),RetentionRate_4yr,SAT Average (Overall),Mean Earnings (6 Yrs after Entry),Mean Earnings (10 Yrs after Entry)
count,1845.000000,1946.000000,1988.000000,1816.000000,1840.000000,1634.000000,1845.000000,1101.000000,1737.000000,1717.000000
mean,64.954835,93293.260021,4499.895875,36848.783590,54.093625,70.366297,74.176428,1143.650318,39203.339090,50188.235294
std,27.416651,32995.291351,8238.877283,17193.376772,20.991844,20.763811,15.305475,130.873861,11386.441444,16223.866961
min,0.000000,6564.000000,0.000000,5663.000000,0.000000,2.440000,0.000000,842.000000,13300.000000,18000.000000
25%,44.740000,73599.000000,638.750000,22808.000000,41.217500,59.662500,67.210000,1054.000000,32700.000000,40800.000000
50%,67.950000,89826.000000,1613.000000,32750.000000,54.615000,74.435000,76.270000,1117.000000,37600.000000,47800.000000
75%,89.380000,110865.000000,4498.500000,48952.750000,68.082500,85.425000,83.650000,1209.000000,43100.000000,55500.000000
max,100.000000,253716.000000,109233.000000,81531.000000,100.000000,100.000000,100.000000,1566.000000,104500.000000,171800.000000


In [13]:
# examine obs with retention rate = 0
df_earn_new[df_earn_new.RetentionRate_4yr==0].head(10)
# retention rate of 0 may be okay for now

,School Name,State,School Ownership,Full-time Faculty Rate (%),Faculty Average Salary,Student Enrollment Size,Attendance Cost,150% Completion Rate at 4 Yr (%),Admission Rate (%),RetentionRate_4yr,Female_Majority,SAT Average (Overall),Mean Earnings (6 Yrs after Entry),Mean Earnings (10 Yrs after Entry)
92,California Christian College,CA,Private NonProfit,NaN,52656.0,13.0,15210.0,NaN,100.00,0.0,False,NaN,NaN,NaN
144,San Francisco Art Institute,CA,Private NonProfit,78.95,94596.0,22.0,70023.0,39.24,94.67,0.0,True,NaN,28400.0,38600.0
613,Cambridge College,MA,Private NonProfit,100.00,91320.0,1071.0,NaN,20.00,NaN,0.0,True,NaN,36400.0,40000.0
767,Cleveland University-Kansas City,KS,Private NonProfit,62.07,67200.0,77.0,NaN,0.00,54.55,0.0,True,NaN,41700.0,53600.0
796,Stevens-The Institute of Business & Arts,MO,Private ForProfit,NaN,58752.0,108.0,22069.0,66.67,NaN,0.0,True,NaN,26100.0,29400.0
1166,Tri-State Bible College,OH,Private NonProfit,NaN,NaN,18.0,14768.0,NaN,NaN,0.0,False,NaN,NaN,NaN
1167,Union Institute & University,OH,Private NonProfit,14.42,66924.0,575.0,31656.0,37.50,NaN,0.0,False,NaN,64200.0,55000.0
1723,Messenger College,TX,Private NonProfit,NaN,32496.0,33.0,22425.0,19.05,33.33,0.0,True,NaN,NaN,23900.0
1724,University of Phoenix-Hawaii,HI,Private ForProfit,NaN,NaN,107.0,18992.0,19.23,NaN,0.0,False,NaN,34200.0,54900.0
1787,United States University,CA,Private ForProfit,88.89,83388.0,116.0,20187.0,0.00,NaN,0.0,True,NaN,52200.0,NaN


In [14]:
# examine obs with enrollment size = 0
# df_earn_new[df_earn_new["Student Enrollment Size"]==0]
# it seems these schools have much of their information missing.

# will therefore remove these obs with enrollment size = 0
clean_dat = df_earn_new[df_earn_new["Student Enrollment Size"]!=0]
clean_dat10 = df_earn_new[df_earn_new["Student Enrollment Size"]!=0]

# df_earn_new[df_earn_new["Student Enrollment Size"]<100]
# NOTE: there are several universities with very low recorded enrollment size


In [15]:
# examine obs with 150% completion rate = 0
df_earn_new[df_earn_new["150% Completion Rate at 4 Yr (%)"]==0].head()
# it seems many obs that could be problematic are missing mean earnings values
# so, will do most important piece first (remove missing values from the response)

,School Name,State,School Ownership,Full-time Faculty Rate (%),Faculty Average Salary,Student Enrollment Size,Attendance Cost,150% Completion Rate at 4 Yr (%),Admission Rate (%),RetentionRate_4yr,Female_Majority,SAT Average (Overall),Mean Earnings (6 Yrs after Entry),Mean Earnings (10 Yrs after Entry)
114,Southern California Seminary,CA,Private NonProfit,33.33,59208.0,50.0,NaN,0.0,NaN,NaN,False,NaN,NaN,NaN
133,Pacific States University,CA,Private NonProfit,NaN,42504.0,2.0,NaN,0.0,NaN,NaN,False,NaN,NaN,NaN
329,American Islamic College,IL,Private NonProfit,NaN,101796.0,5.0,NaN,0.0,NaN,NaN,False,NaN,NaN,NaN
465,Palmer College of Chiropractic,IA,Private NonProfit,81.13,75096.0,29.0,NaN,0.0,NaN,NaN,False,NaN,43800.0,53600.0
767,Cleveland University-Kansas City,KS,Private NonProfit,62.07,67200.0,77.0,NaN,0.0,54.55,0.0,True,NaN,41700.0,53600.0


In [16]:
# drop obs with missing values in response column(s)
clean_dat = clean_dat[clean_dat["Mean Earnings (6 Yrs after Entry)"].isnull()==False]
# 250 rows removed => leaving 1736 remaining

In [17]:
# check where Mean Earnings of 10 years post entry is missing
# and drop them in clean_dat10 file
clean_dat10 = clean_dat10[clean_dat10["Mean Earnings (10 Yrs after Entry)"].isnull()==False]
# 1716 obs remaining

In [18]:
# clean up variable names (i.e., column names)
clean_dat.rename(columns = {"RetentionRate_4yr": "Retention Rate",
                            "150% Completion Rate at 4 Yr (%)": "Completion Rate",
                            "SAT Average (Overall)": "SAT Average"},
                inplace=True)

clean_dat10 = clean_dat10.rename(columns = {"RetentionRate_4yr": "Retention Rate",
                            "150% Completion Rate at 4 Yr (%)": "Completion Rate",
                            "SAT Average (Overall)": "SAT Average"})

In [24]:
# set index as School Name
clean_dat.index = clean_dat["School Name"]
clean_dat10.index = clean_dat10["School Name"]
# check that indexs are the same
print(clean_dat.index[0:10])
print(clean_dat10.index[0:10])

Index(['Alabama A & M University', 'University of Alabama at Birmingham',
       'University of Alabama in Huntsville', 'Alabama State University',
       'The University of Alabama', 'Athens State University',
       'Auburn University at Montgomery', 'Auburn University',
       'Birmingham-Southern College', 'South University-Montgomery'],
      dtype='object', name='School Name')
Index(['Alabama A & M University', 'University of Alabama at Birmingham',
       'University of Alabama in Huntsville', 'Alabama State University',
       'The University of Alabama', 'Athens State University',
       'Auburn University at Montgomery', 'Auburn University',
       'Birmingham-Southern College', 'South University-Montgomery'],
      dtype='object', name='School Name')


In [72]:
# examine missing values prevalance further (after missing response values (for 6yr earnings) have been removed)
missing_values = clean_dat.isnull().sum()
print(missing_values.sort_values(ascending=False))
missing_values_in_rows = clean_dat.isnull().sum(axis=1)
print(missing_values_in_rows.sort_values(ascending=False))
# important to consider: what is good threshold of missing values to constitue throwing out obs?
# could also try EM algorithm + MI for imputing these data
print(missing_values_in_rows[missing_values_in_rows>6]) # 3 obs
# appears from EDA that alot of private for profit universites do not include their SAT scores
# hope to overcome this with KNN Imputer

SAT Average                           676
Admission Rate (%)                    260
Attendance Cost                        84
Retention Rate                         79
Completion Rate                        71
Full-time Faculty Rate (%)             36
Mean Earnings (10 Yrs after Entry)     29
Faculty Average Salary                 16
Student Enrollment Size                 1
School Name                             0
State                                   0
School Ownership                        0
Female_Majority                         0
Mean Earnings (6 Yrs after Entry)       0
dtype: int64
School Name
Excelsior College                                      8
Thomas Edison State University                         7
Careers Unlimited                                      7
California Institute of Integral Studies               6
MGH Institute of Health Professions                    6
                                                      ..
Delta State University                       

In [73]:
# python list of ivy league + plus ivy league plus schools,
# which BING AI (CHAT GPT) helped me generate
ivy_plus = [
    "Brown University",
    "Columbia University in the City of New York",
    "Cornell University",
    "Dartmouth College",
    "Harvard University",
    "University of Pennsylvania",
    "Princeton University",
    "Yale University",
    "Massachusetts Institute of Technology",
    "Stanford University",
    "California Institute of Technology",
    "Duke University",
    "Johns Hopkins University",
    "Northwestern University",
    "University of Chicago"
]

In [74]:
# add variable regarding
clean_dat["Ivy League Plus"] = clean_dat["School Name"].isin(ivy_plus).astype(int)
clean_dat10["Ivy League Plus"] = clean_dat10["School Name"].isin(ivy_plus).astype(int)
clean_dat[clean_dat["Ivy League Plus"]==1]

,School Name,State,School Ownership,Full-time Faculty Rate (%),Faculty Average Salary,Student Enrollment Size,Attendance Cost,Completion Rate,Admission Rate (%),Retention Rate,Female_Majority,SAT Average,Mean Earnings (6 Yrs after Entry),Mean Earnings (10 Yrs after Entry),Ivy League Plus
School Name,,,,,,,,,,,,,,,
California Institute of Technology,California Institute of Technology,CA,Private NonProfit,93.78,242736.0,901.0,74763.0,91.56,6.69,93.62,False,1566.0,68500.0,106300.0,1
Yale University,Yale University,CT,Private NonProfit,72.33,233880.0,4701.0,76645.0,96.39,6.53,65.42,False,1520.0,67800.0,124400.0,1
University of Chicago,University of Chicago,IL,Private NonProfit,80.18,221184.0,7010.0,81531.0,95.98,7.31,99.65,False,1537.0,72100.0,103000.0,1
Northwestern University,Northwestern University,IL,Private NonProfit,83.53,210156.0,8401.0,78654.0,94.95,9.31,96.96,True,1505.0,71900.0,93400.0,1
Johns Hopkins University,Johns Hopkins University,MD,Private NonProfit,97.26,185004.0,5752.0,74001.0,94.40,11.06,96.51,True,1532.0,68000.0,89300.0,1
Harvard University,Harvard University,MA,Private NonProfit,86.96,253716.0,6099.0,75914.0,97.63,5.01,75.67,True,1520.0,91300.0,139100.0,1
Massachusetts Institute of Technology,Massachusetts Institute of Technology,MA,Private NonProfit,98.86,235680.0,4360.0,73160.0,95.58,7.26,97.64,False,1550.0,99600.0,153600.0,1
Dartmouth College,Dartmouth College,NH,Private NonProfit,82.63,187200.0,4146.0,77152.0,94.86,9.22,96.13,False,1500.0,74600.0,110200.0,1
Princeton University,Princeton University,NJ,Private NonProfit,82.77,250020.0,4688.0,74150.0,98.02,5.63,83.30,True,1506.0,73600.0,116300.0,1


In [75]:
# save data
clean_dat.to_csv("./saved_data/college_roi_dat_clean.csv",index=False)
clean_dat10.to_csv("./saved_data/college_roi_dat_clean10.csv",index=False)

### Preprocessing

In [76]:
# define features and target
features = ["School Ownership", 'Full-time Faculty Rate (%)', 'Faculty Average Salary', 'Student Enrollment Size','Attendance Cost',
            'Completion Rate', 'Admission Rate (%)', 'Retention Rate','SAT Average', 'Ivy League Plus']
X = clean_dat.loc[:,features]
X10 = clean_dat10.loc[:,features]
# X.index = clean_dat['School Name']
target = clean_dat['Mean Earnings (6 Yrs after Entry)']
target2 = clean_dat10['Mean Earnings (10 Yrs after Entry)']

# could add target 3, etc for median, low med high income earnings
# could be cool for model such as "Schools that produce highest expected earnings for low income students, and why"
# obviously, could look at individual schools in the aggregate, but having fitted model helps show overall characteristics;
# ===> can provide insights into specific characteristics that lead to higher/lower expected income of students

Xtrain, Xtest0, ytrain, ytest0 = train_test_split(X,target,random_state=4015,test_size=0.3)
# Xvalid,Xtest,yvalid,ytest = train_test_split(Xtest0,ytest0, random_state=4015,test_size=0.3)

# 10 year
Xtrain10, Xtest10, ytrain10, ytest10 = train_test_split(X10,target2,random_state=4015,test_size=0.3)

In [79]:
# any missing Ivy League values?
any(X10["Ivy League Plus"].isnull())

False

In [80]:
# define preprocessing
numeric_features = X.select_dtypes(exclude=['object']).columns
categorical_features = ['School Ownership']

numeric_transformer = Pipeline([
    #("imputer", SimpleImputer(strategy="mean")),
    # try switching this to KNNImpter(n_neighbors = 6, weights = "distance",add_indicator = True)
    ("imputer", KNNImputer(n_neighbors = 6, weights = "distance")),
    # ("poly2", PolynomialFeatures(degree=2)),
    # ("scaler", StandardScaler()) # not going to scale for now, to keep as much interpretability as possible
    # also, important to be careful with scaling 0, 1 vars (and probably don't want to scale them)
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [87]:
# run preprocessor 
X_filled= preprocessor.fit_transform(X)
Xtrain_filled = preprocessor.fit_transform(Xtrain)
Xtest_filled= preprocessor.fit_transform(Xtest0)
X_filled10= preprocessor.fit_transform(X10)
Xtrain_filled10 = preprocessor.fit_transform(Xtrain10)
Xtest_filled10= preprocessor.fit_transform(Xtest10)

# get feature names from one hot encoder
ohenc_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out()

feature_names = list(X.columns)[1:]+list(ohenc_names)
print(feature_names)

display(pd.DataFrame(X_filled))
print(X_filled.shape)
# redefine dataframes with appropriate column names
# get indexes from train and test and apply them as well
X_filled = pd.DataFrame(X_filled, columns=feature_names,index = X.index)
Xtrain_filled = pd.DataFrame(Xtrain_filled, columns=feature_names,index = Xtrain.index)
Xtest_filled = pd.DataFrame(Xtest_filled, columns=feature_names,index = Xtest0.index)
X_filled10 = pd.DataFrame(X_filled10, columns=feature_names,index = X10.index)
Xtrain_filled10 = pd.DataFrame(Xtrain_filled10, columns=feature_names,index = Xtrain10.index)
Xtest_filled10 = pd.DataFrame(Xtest_filled10, columns=feature_names,index = Xtest10.index)
display(X_filled10)
X_filled10.shape

['Full-time Faculty Rate (%)', 'Faculty Average Salary', 'Student Enrollment Size', 'Attendance Cost', 'Completion Rate', 'Admission Rate (%)', 'Retention Rate', 'SAT Average', 'Ivy League Plus', 'x0_Private ForProfit', 'x0_Private NonProfit', 'x0_Public']


,0,1,2,3,4,5,6,7,8,9,10,11
0,99.60,91188.000000,5090.0,23445.000000,28.66,89.650000,54.030000,959.000000,0.0,0.0,0.0,1.0
1,76.19,136560.000000,13549.0,25542.000000,61.17,80.600000,86.400000,1245.000000,0.0,0.0,0.0,1.0
2,67.02,116364.000000,7825.0,24861.000000,57.14,77.110000,81.800000,1300.000000,0.0,0.0,0.0,1.0
3,67.97,86328.000000,3603.0,21892.000000,31.77,98.880000,62.020000,938.000000,0.0,0.0,0.0,1.0
4,77.07,124188.000000,30610.0,30016.000000,72.14,80.390000,87.230000,1262.000000,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1731,3.43,250836.000000,2363.0,18184.000000,24.81,59.442951,33.330000,1484.761267,0.0,1.0,0.0,0.0
1732,65.27,130944.000000,127.0,30311.358106,8.70,83.647959,41.652194,1023.041635,0.0,1.0,0.0,0.0
1733,2.33,93671.790509,228.0,16367.000000,12.50,50.570252,50.000000,980.340823,0.0,1.0,0.0,0.0
1734,100.00,100452.000000,3357.0,18930.000000,36.52,92.990000,70.770000,956.000000,0.0,0.0,0.0,1.0


(1736, 12)


,Full-time Faculty Rate (%),Faculty Average Salary,Student Enrollment Size,Attendance Cost,Completion Rate,Admission Rate (%),Retention Rate,SAT Average,Ivy League Plus,x0_Private ForProfit,x0_Private NonProfit,x0_Public
School Name,,,,,,,,,,,,
Alabama A & M University,99.60,91188.000000,5090.0,23445.000000,28.66,89.650000,54.030000,959.000000,0.0,0.0,0.0,1.0
University of Alabama at Birmingham,76.19,136560.000000,13549.0,25542.000000,61.17,80.600000,86.400000,1245.000000,0.0,0.0,0.0,1.0
University of Alabama in Huntsville,67.02,116364.000000,7825.0,24861.000000,57.14,77.110000,81.800000,1300.000000,0.0,0.0,0.0,1.0
Alabama State University,67.97,86328.000000,3603.0,21892.000000,31.77,98.880000,62.020000,938.000000,0.0,0.0,0.0,1.0
The University of Alabama,77.07,124188.000000,30610.0,30016.000000,72.14,80.390000,87.230000,1262.000000,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
University of Phoenix-California,3.43,250836.000000,2363.0,18184.000000,24.81,70.234086,33.330000,1484.761267,0.0,1.0,0.0,0.0
University of Phoenix-Nevada,65.27,130944.000000,127.0,25839.134522,8.70,72.027395,56.636516,1023.041635,0.0,1.0,0.0,0.0
University of Phoenix-Texas,2.33,93671.790509,228.0,16367.000000,12.50,50.570252,50.000000,980.340823,0.0,1.0,0.0,0.0


(1716, 12)

In [89]:
# export preprocessed and split data!
X_filled.to_csv("./saved_data/X_filled.csv")
Xtrain_filled.to_csv("./saved_data/Xtrain_filled.csv")
Xtest_filled.to_csv("./saved_data/Xtest_filled.csv")
ytrain.to_csv("./saved_data/ytrain.csv")
ytest0.to_csv("./saved_data/ytest.csv")
# 10 year post entry data
X_filled10.to_csv("./saved_data/X_filled10.csv")
Xtrain_filled10.to_csv("./saved_data/Xtrain_filled10.csv")
Xtest_filled10.to_csv("./saved_data/Xtest_filled10.csv")
ytrain10.to_csv("./saved_data/ytrain10.csv")
ytest10.to_csv("./saved_data/ytest10.csv")
print("cleaned data exported. :)")

cleaned data exported. :)
